# UNR - FCEIA 
## Tecnicatura Universitaria en Programación 
### NLP: Trabajo Práctico N°1 

---

**Integrantes**
- López Ceratto, Julieta : L-3311/1
- Crenna, Giuliano : C-7438/1

# Importamos librerías necesarias

In [100]:
import os
import pandas as pd
import pickle
from typing import List, Dict, Any
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
import tensorflow as tf
import tensorflow_text
import tensorflow_hub as hub
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [101]:
import warnings
warnings.filterwarnings("ignore")

# Carga de datasets

In [102]:
JUEGOS_PATH: str = os.path.join(os.getcwd(), 'data', 'bgg_database.csv')
PELICULAS_PATH: str = os.path.join(os.getcwd(), 'data', 'IMDB-Movie-Data.csv')
LIBROS_PATH: str = os.path.join(os.getcwd(), 'data', 'dataset_libros.csv')

In [103]:
dataset_juegos: pd.DataFrame = pd.read_csv(JUEGOS_PATH)
dataset_peliculas: pd.DataFrame = pd.read_csv(PELICULAS_PATH)
dataset_libros: pd.DataFrame = pd.read_csv(LIBROS_PATH)

# Modelo de Análisis de Sentimientos

Creo un dataset sencillo para entrenar al clasificador.

In [104]:
ESTADOS_ANIMO_PATH: str = os.path.join(os.getcwd(), 'data', 'estados_de_animo.csv')

In [105]:
df_estados_de_animo: pd.DataFrame = pd.read_csv(ESTADOS_ANIMO_PATH)

X: pd.Series = df_estados_de_animo['prompt']
y: pd.Series = df_estados_de_animo['estado_animo']

Hacemos un split de los datos y creamos un pipeline de trabajo utilizando el clasificador **MultinomialNB**.

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

modelo_animo = make_pipeline(TfidfVectorizer(), MultinomialNB())

In [107]:
modelo_animo.fit(X_train, y_train)

y_pred = modelo_animo.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      Alegre       1.00      0.96      0.98        79
 Melancólico       1.00      1.00      1.00        65
 Ni fu ni fa       0.96      1.00      0.98        76

    accuracy                           0.99       220
   macro avg       0.99      0.99      0.99       220
weighted avg       0.99      0.99      0.99       220



Creamos una función para clasificar el prompt del usuario.

In [108]:
def clasificar_animo(prompt_usuario: str) -> str:
    estado_animo_predicho = modelo_animo.predict([prompt_usuario])[0]
    
    return estado_animo_predicho

In [109]:
nuevo_prompt = "La vida no significa nada"

print(f"Estado de ánimo: {clasificar_animo(nuevo_prompt)}")

Estado de ánimo: Ni fu ni fa


In [110]:
nuevo_prompt = "Me siento feliz"

print(f"Estado de ánimo: {clasificar_animo(nuevo_prompt)}")

Estado de ánimo: Alegre


In [111]:
nuevo_prompt = "El vacio se siente en mi"

print(f"Estado de ánimo: {clasificar_animo(nuevo_prompt)}")

Estado de ánimo: Melancólico


Exporto el modelo

In [112]:
# ESTADO_ANIMO_MODEL_PATH: str = os.path.join(os.getcwd(), 'models', 'modelo_estado_animo.pickle')

# pickle.dump(modelo_animo, open(ESTADO_ANIMO_MODEL_PATH, 'wb'))

# Análisis Datasets

## Análisis Dataset Juegos
Como Vemos en el dataset de juegos no se presentan datos nulos.

In [113]:
dataset_juegos.isna().sum()

rank                0
game_name           0
game_href           0
geek_rating         0
avg_rating          0
num_voters          0
description         0
yearpublished       0
minplayers          0
maxplayers          0
minplaytime         0
maxplaytime         0
minage              0
avgweight           0
best_num_players    0
designers           0
mechanics           0
categories          0
dtype: int64

## Análisis Dataset Películas

In [114]:
dataset_peliculas.isna().sum()

Rank                  0
Title                 0
Genre                 0
Description           0
Director              0
Actors                0
Year                  0
Runtime (Minutes)     0
Rating                0
Votes                 0
Revenue (Millions)    0
Metascore             0
dtype: int64

## Dataset Libros
De los 5984 libros dentro del dataset, solo 53 tiene titulo secundario y solo 2796 libros tienen un autor registrado.

In [115]:
dataset_libros.shape

(5984, 4)

In [116]:
dataset_libros.isna().sum()

Titulo Principal        0
Titulo Secundario    5931
Autor                3188
N° Ref                  0
dtype: int64

# Apartado 1

## Implementación modelo KNN con encoder de tensorflow

In [117]:
# Cargar Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [204]:
def crear_embeding():
    '''
    Crea dataset de embedings para entrenar el modelo y realizar posterior búsqueda
    en los resultados
    '''
    ###Agrega index y tipo a los dataset para luego recuperar el vecino más cercano
    ### por índice.
    dataset_juegos['index'] = [i for i in dataset_juegos.index]
    dataset_juegos['tipo'] = 'juego'
    dataset_libros['index'] = [i for i in dataset_libros.index]
    dataset_libros['tipo'] = 'libro'
    dataset_peliculas['index'] = [i for i in dataset_peliculas.index]
    dataset_peliculas['tipo'] = 'pelicula'

    ###Agrega una columna 'frase_embedding' que junta la información que se utilizará para entrenar
    ### al modelo.
    
    # Para dataset_juegos
    dataset_juegos['frase_embeding'] = dataset_juegos.apply(
        lambda row: f"{row['description']}, tipo juego, {row['maxplayers']}", axis=1)

    # Para dataset_peliculas
    dataset_peliculas['frase_embeding'] = dataset_peliculas.apply(
        lambda row: f"{row['Description']}, tipo pelicula, {row['Genre']}", axis=1)

    # Para dataset_libros
    dataset_libros['frase_embeding'] = dataset_libros.apply(
        lambda row: f"{row['Titulo Principal']}, tipo libro, {row['Autor']}", axis=1)
    
    # Generar embeddings para cada conjunto de datos
    embeding_juegos = embed(dataset_juegos['frase_embeding']).numpy()
    embeding_libros = embed(dataset_libros['frase_embeding']).numpy()
    embeding_peliculas = embed(dataset_peliculas['frase_embeding']).numpy()

    # Crear DataFrames para los embeddings
    embeding_juegos_df = pd.DataFrame(embeding_juegos)
    embeding_libros_df = pd.DataFrame(embeding_libros)
    embeding_peliculas_df = pd.DataFrame(embeding_peliculas)

    # Añadir el índice y tipo a los DataFrames de embeddings
    embeding_juegos_df['index'] = dataset_juegos['index']
    embeding_libros_df['index'] = dataset_libros['index']
    embeding_peliculas_df['index'] = dataset_peliculas['index']
    embeding_juegos_df['tipo'] = dataset_juegos['tipo']
    embeding_libros_df['tipo'] = dataset_libros['tipo']
    embeding_peliculas_df['tipo'] = dataset_peliculas['tipo'] 

    # Concatenar todos los embeddings
    embedings_totales = np.concatenate([embeding_juegos, embeding_libros, embeding_peliculas])
    df_embedings_totales = pd.concat([embeding_juegos_df, embeding_libros_df, embeding_peliculas_df])
    
    # Guardar el DataFrame de embeddings en un archivo CSV
    df_embedings_totales.to_csv('./data/embedings_totales.csv', index=False)




In [206]:
#crear_embeding()
df_embedings_totales = pd.read_csv('./data/embedings_totales.csv')

In [228]:
n_neighbors = 5
modelor_recomendador = make_pipeline(NearestNeighbors(n_neighbors=n_neighbors, metric='cosine', algorithm='brute'))
modelor_recomendador.fit(df_embedings_totales.drop(columns=['index', 'tipo']))

Pipeline(steps=[('nearestneighbors',
                 NearestNeighbors(algorithm='brute', metric='cosine'))])

In [229]:
RECOMENDADOR_MODEL_PATH: str = os.path.join(os.getcwd(), 'models', 'modelo_recomendador.pickle')

pickle.dump(modelor_recomendador, open(RECOMENDADOR_MODEL_PATH, 'wb'))

In [230]:
animo = clasificar_animo('Quiero estar tranquilo')

In [231]:
user_prompt = f'historia de amor en la playa, {animo}'

In [236]:
def que_hacer (consulta : str):
    '''
    Devuelve 5 recomendaciones más acordes a la consulta
    '''
    consulta = embed(consulta).numpy()

    # Realizar la búsqueda de los vecinos más cercanos
    distances, indices = modelor_recomendador[0].kneighbors(consulta)

    for j in range(n_neighbors):
        # Obtenemos el índice del vecino más cercano
        idx = indices[0][j]
        i = df_embedings_totales['index'].iloc[idx]
        dataset = df_embedings_totales['tipo'].iloc[idx]

        # Dependiendo del dataset, accedemos al DataFrame correspondiente
        if dataset == 'juego':
            vecino = dataset_juegos['game_name'].iloc[i]
        elif dataset == 'libro':
            vecino = dataset_libros['Titulo Principal'].iloc[i]
        elif dataset == 'peliculas':
            vecino = dataset_peliculas['Title'].iloc[i]
        
        # Imprimir el vecino y la distancia
        print(f"Vecino {j + 1}: {vecino} - Distancia: {distances[0][j]:.4f} - {dataset}")

    print("-" * 40)

In [237]:
que_hacer(user_prompt)

Vecino 1: Love for Love: A Comedy - Distancia: 0.5112 - libro
Vecino 2: El arte de amar - Distancia: 0.5347 - libro
Vecino 3: El arte de amar - Distancia: 0.5347 - libro
Vecino 4: El arte de amar - Distancia: 0.5347 - libro
Vecino 5: Ars Amatoria; or, The Art Of Love - Distancia: 0.5559 - libro
----------------------------------------
